In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import json

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all

## collect campaign data

In [ ]:
#read sheets
ce_wpwp           = pd.read_csv('raw_data/campaigns_editors_july_wpwp.csv', parse_dates=True)
ce_dashboard      = pd.read_csv('raw_data/campaigns_editors_dashboard.csv', parse_dates=True)
ce_wikiquote      = pd.read_csv('raw_data/campaigns_editors - Projet Wikiquote Tour 2020.csv', parse_dates=True)
ce_aflia_1lib1ref = pd.read_csv('raw_data/campaigns_editors - AfLIA 1Lib1Ref African Librarians_PEashboard.csv', parse_dates=True)
ce_afroCine       = pd.read_csv('raw_data/campaigns_editors - Project_AfroCine_Months_of_African_Cinema.csv', parse_dates=True)
ce_lagos          = pd.read_csv('raw_data/campaigns_editors - Lagos Municipalities Photowalk.csv', parse_dates=True)
ce_1lib1ref       = pd.read_csv('raw_data/campaigns_editors - 1lib1ref.csv', parse_dates=True)

#read wiki loves data
f = open('raw_data/db.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
#clean sheets

#make all column names lower case
ce_dashboard.columns= ce_dashboard.columns.str.lower()
ce_wikiquote.columns= ce_wikiquote.columns.str.lower()
ce_afroCine.columns= ce_afroCine.columns.str.lower()
ce_lagos.columns= ce_lagos.columns.str.lower()
ce_1lib1ref.columns= ce_1lib1ref.columns.str.lower()
ce_aflia_1lib1ref.columns= ce_aflia_1lib1ref.columns.str.lower()
ce_wpwp.columns= ce_wpwp.columns.str.lower()

#add a column to each sheet with the name of the campaign
ce_wpwp['campaign']           = 'wpwp'
ce_wikiquote['campaign']      = 'Projet_Wikiquote_Tour'
ce_afroCine['campaign']       = 'Project_AfroCine_Months_of_African_Cinema'
ce_lagos['campaign']          = 'Lagos_Municipalities_Photowalk'
ce_aflia_1lib1ref['campaign'] = 'AfLIA_1Lib1Ref_African_Librarians'
ce_1lib1ref['campaign']       = '1lib1ref'

#rename course to campaign
ce_dashboard = ce_dashboard.rename(columns={'course':'campaign','timestamp':'date'})
ce_wpwp      = ce_wpwp.rename(columns={'timestamp':'date'})
ce_lagos     = ce_lagos.rename(columns={'end':'date'})
ce_1lib1ref  = ce_1lib1ref.rename(columns={'timestamp':'date'})

#add date column 
ce_aflia_1lib1ref['date'] = '2020' #empty date initially
ce_wikiquote['date'] = '2020'

ce_dashboard['source']     = 'dashboard'
ce_wpwp['source']           = 'wpwp'
ce_wikiquote['source']      = 'Projet_Wikiquote_Tour'
ce_afroCine['source']       = 'Project_AfroCine_Months_of_African_Cinema'
ce_lagos['source']          = 'Lagos_Municipalities_Photowalk'
ce_aflia_1lib1ref['source'] = 'AfLIA_1Lib1Ref_African_Librarians'
ce_1lib1ref['source']       = '1lib1ref'

#select the columns I am interested in
ce_dashboard_s = ce_dashboard[['date', 'campaign', 'username', 'source']] 
ce_wpwp_s = ce_wpwp[['date', 'campaign', 'username', 'source']] 
ce_wikiquote_s = ce_wikiquote[['date', 'campaign', 'username', 'source']] 
ce_afroCine_s = ce_afroCine[['date', 'campaign', 'username','source']] 
ce_lagos_s = ce_lagos[['date', 'campaign', 'username','source']] 
ce_aflia_1lib1ref_s = ce_aflia_1lib1ref[['date', 'campaign', 'username','source']] 
ce_1lib1ref_s = ce_1lib1ref[['date', 'campaign', 'username','source']] 

#handle date columns
ce_dashboard_s['date']      = pd.to_datetime(ce_dashboard_s['date']).dt.tz_localize(None).dt.strftime('%Y-%m-%d')
ce_wpwp_s['date']           = pd.to_datetime(ce_wpwp_s['date']).dt.tz_localize(None).dt.strftime('%Y-%m-%d')
ce_1lib1ref_s['date']       = pd.to_datetime(ce_1lib1ref_s['date']).dt.tz_localize(None).dt.strftime('%Y-%m-%d')
ce_aflia_1lib1ref_s['date'] = pd.to_datetime(ce_aflia_1lib1ref_s['date']).dt.strftime('%Y-%m-%d')
ce_wikiquote_s['date']      = pd.to_datetime(ce_wikiquote_s['date']).dt.strftime('%Y-%m-%d')
ce_afroCine_s['date']       = pd.to_datetime(ce_afroCine_s['date']).dt.strftime('%Y-%m-%d')
ce_lagos_s['date']          = pd.to_datetime(ce_lagos_s['date']).dt.strftime('%Y-%m-%d')  

#combine
ce_d = pd.concat([ce_aflia_1lib1ref_s, 
                  ce_wikiquote_s,
                  ce_afroCine_s,
                  ce_lagos_s,
                  ce_1lib1ref_s,
                  ce_wpwp_s, 
                  ce_dashboard_s]) 

In [ ]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [ ]:
# clean json

# converting json dataset from dictionary to dataframe
train = pd.DataFrame.from_dict(data, orient='index')
train.reset_index(level=0, inplace=True)

        # add index column to list for below selection
        # ssa_ls = ssa.insert(0, "index")
        #ssa_ls = np.append(ssa, ['index'])

        # Select the ones you want
        #df_new = train.reindex(columns = ssa_ls)#, fill_value=0)

        # put index at first place
        #col = df_new.pop("index")
        #df_new.insert(0, col.name, col)

# reshape 
jdf = (train.iloc[1:].set_index('index').stack().reset_index(0).join(train.iloc[0,1:].rename('1')).sort_values('index'))

ce_wikiloves= pd.DataFrame([flatten_json(x) for x in jdf[0]])

#keep cols
keep_cols = [col for col in ce_wikiloves.columns if 'users' in col]
#drop _usage, _reg
drop_usage_cols = [col for col in ce_wikiloves.columns if '_usage' in col]
drop_reg_cols = [col for col in ce_wikiloves.columns if '_reg' in col]
drop_data_cols = [col for col in ce_wikiloves.columns if 'data_' in col]
drop_cols = drop_usage_cols +drop_reg_cols+drop_data_cols

# Select the columns desired
ce_wikiloves_s = ce_wikiloves.drop(columns=drop_cols)

#reshape wide to long
#as each row is essentially it's own df
keys = [c for c in ce_wikiloves_s if c.startswith('users_')]
ce_wikiloves_sdf = pd.melt(ce_wikiloves_s, id_vars=['category','end','count'], value_vars=keys, value_name='key')
#remove "users_" and "_count" from column names
ce_wikiloves_sdf['variable'] = ce_wikiloves_sdf['variable'].map(lambda x: x.lstrip('users_').rstrip('_count'))




ce_wikiloves_sdf = ce_wikiloves_sdf.rename(columns={'category':'campaign',
                                                    'end':'date',
                                                    'variable':'username',
                                                    'key': 'users_uploads_count',
                                                    'count': 'total_uploads_count'
                                                   })
ce_wikiloves_sdf['source'] = 'wikiloves_json'

#drop rows of Pandas DataFrame whose value in a certain column is NaN
ce_wikiloves_sdf = ce_wikiloves_sdf[ce_wikiloves_sdf['users_uploads_count'].notna()]

#handle date column
#ce_wikiloves_sdf['date'] = pd.to_datetime(ce_wikiloves_sdf['date']).dt.strftime('%Y-%m-%d') 
ce_wikiloves_sdf['date'] = pd.to_datetime(ce_wikiloves_sdf.date, format='%Y%m%d%H%M%S', errors='coerce').dt.strftime('%Y-%m-%d') 

## combine campaign dfs

In [ ]:
#concatenate all usernames/dates/campaigns
campaigns_combined = pd.concat([ce_d, ce_wikiloves_sdf]) 

In [ ]:
# Filter data between two dates
campaigns_combined_select = campaigns_combined.loc[(campaigns_combined['date'] >= '2018-12-31')
                     & (campaigns_combined['date'] < '2021-01-01')]

In [ ]:
editors_unique = campaigns_combined['username'].unique()
editors_unique_s = campaigns_combined_select['username'].unique()
print(len(editors_unique))
print(len(editors_unique_s))

### final campaign dataframe

In [ ]:
table_data = campaigns_combined_select[['username', 'campaign', 'source']]

In [ ]:
table_data.to_csv("raw_data/campaign_participants_20192020.csv", sep=',', encoding = 'utf-8', index=False)

## Geoeditors

In [ ]:
#First
#wmf_raw.mediawiki_user
#map your user_name data to wiki_db and user_id 

#Second
#join that with wiki_db and user_fingerprint_or_id to wmf.editors_daily to identify locations.


In [ ]:
#https://www.mediawiki.org/wiki/Manual:CategoryMembershipChanges

editors_slow = hive.run("""
WITH user_names AS (
    SELECT DISTINCT username
    FROM florez.campaign_participants_20192020
),

geo_users AS (
    SELECT wiki_db, user_fingerprint_or_id, country_code
    FROM wmf.editors_daily
    WHERE editors_daily.month = '2021-09'
        AND user_is_anonymous = false
        AND action_type IN (0, 1, 6) 
)

SELECT userw.user_name, 
        userw.wiki_db, 
        userw.user_registration, 
        userw.user_editcount, 
        ed.user_fingerprint_or_id,
        ed.country_code
FROM user_names
JOIN wmf_raw.mediawiki_user userw
  ON user_names.username = userw.user_name
JOIN geo_users ed
  ON ed.user_fingerprint_or_id = userw.user_id
  AND ed.wiki_db = userw.wiki_db
WHERE userw.snapshot = '2021-09'
""")

In [ ]:
editors_slow.drop_duplicates().info()

In [ ]:
editors_slow.to_csv("output_data/output.csv", sep=',', encoding = 'utf-8', index=False)

## Geoeditors

In [ ]:
#import findspark
from pyspark.sql.types import ArrayType, StringType

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
#--AND userw.user_editcount > 0

# Gather 
e_locations_r =  '''
WITH user_names AS (
    SELECT DISTINCT username
    FROM florez.campaign_participants_20192020
),

geo_users AS (
    SELECT wiki_db, user_fingerprint_or_id, country_code
    FROM wmf.editors_daily
    WHERE editors_daily.month = '2021-09'
        AND user_is_anonymous = false
        AND action_type IN (0, 1, 6)
)

SELECT userw.user_name, 
        userw.wiki_db, 
        userw.user_registration, 
        userw.user_editcount, 
        ed.user_fingerprint_or_id,
        ed.country_code
FROM user_names
JOIN wmf_raw.mediawiki_user userw
  ON user_names.username = userw.user_name
JOIN geo_users ed
  ON ed.user_fingerprint_or_id = userw.user_id
  AND ed.wiki_db = userw.wiki_db
WHERE userw.snapshot = '2021-09'
LIMIT 5
'''

In [ ]:
data = spark.sql(e_locations_r = spark.sql(e_locations_r).toPandas()s_r).toPandas()

In [ ]:
african_countries = hive.run("""
SELECT name AS country_name, iso_code AS country_code
FROM canonical_data.countries 
WHERE maxmind_continent = 'Africa'
""")

In [ ]:
#regions
regions = pd.read_csv('raw_data/cr_regions_list.csv')
regions.Region.fillna(value=np.nan, inplace=True)

regions = regions.rename(columns={'Country':'country_name'})

african_countries_w_ssa = african_countries.merge(regions, how='left', on='country_name')
african_countries_w_ssa = african_countries_w_ssa[['country_name', 'country_code', 'Region']]

In [ ]:
african_countries_w_ssa.to_csv("output_data/african_countries_w_ssa.csv", sep=',', encoding = 'utf-8', index=False)

In [ ]:
### Current Africa GeoEditors

In [ ]:
#https://www.mediawiki.org/wiki/Manual:CategoryMembershipChanges

editors_ssa = hive.run("""
SELECT SUM(distinct_editors), c.name
FROM wmf.geoeditors_monthly AS g
JOIN canonical_data.countries AS c
ON c.iso_code  = g.country_code 
WHERE c.maxmind_continent ='Africa'
AND c.name IN (SSA_countries) 
GROUP BY c.name
""")

In [ ]:
editors_ssa.to_csv("output_data/editors_ssa.csv", sep=',', encoding = 'utf-8', index=False)